## Handwritten Dzongkha Alphbate Recognition Using CNN

### Mount Google Drive

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import the libraries

In [30]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization, Activation
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
from keras import backend as K

In [31]:
DATA_PATH = "/content/drive/MyDrive/GCIT/PRJ303/Canvas_Dataset /"

# Variable Declaration

In [32]:
img_width, img_height = 32, 32

# Determine Input Size

In [33]:
if K.image_data_format() == 'channels_first':
  input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### Data Generator

In [34]:
train_datagen = ImageDataGenerator( # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
  rescale = 1./255,
  shear_range=0.1,
  rotation_range=5, 
    zoom_range = 0.1,
    width_shift_range=0.1, 
    height_shift_range=0.1, 
)

test_datagen = ImageDataGenerator(
   rescale = 1./255 
)

### Load images

In [35]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory
train_data = train_datagen.flow_from_directory(
  DATA_PATH+'Train',
  target_size = (img_width, img_height),
  class_mode = 'categorical',
  batch_size = 32,
  shuffle = True
)

test_data = train_datagen.flow_from_directory(
  DATA_PATH+'Test',
  target_size = (img_width, img_height),
  class_mode = 'categorical',
  batch_size = 32,
)

Found 1981 images belonging to 30 classes.
Found 450 images belonging to 30 classes.


### Build the Model

* CONV->ReLU-> MaxPool
* CONV->ReLU-> MaxPool
* CONV->ReLU-> MaxPool
* CONV->ReLU-> MaxPool
* CONV->ReLU-> MaxPool

In [36]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same',
                 activation ='relu', input_shape = (input_shape)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    
model.add(Conv2D(filters =128, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(30))
model.add(Activation('sigmoid'))

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 32, 32, 32)        2432      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 4, 4, 64)        

In [38]:
# Model Callback - Stop the training when no learning is taking place
# Callback is object that is used to do certain task in model fitting
# Here callback is used as Earling Stopping Object in order to check weather there is no improvement model loss
# The Model fitting stops when there is no change in loss after  3 epochs
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'loss', 
    patience = 3,
    verbose = 0,
    mode = "min"
    )

### Compile the model

In [39]:
# Compile the model
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

### Fit the model

In [40]:
model.fit(
    train_data,
    # Number of batches for training
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    # Number of batches for testing
    validation_steps = len(test_data),
    epochs = 50,
    
    callbacks = [callback],
    verbose = 1,
)

Epoch 1/50
62/62 [==============================] - 642s 10s/step - loss: 3.4021 - accuracy: 0.0429 - val_loss: 3.3991 - val_accuracy: 0.0511
Epoch 2/50
62/62 [==============================] - 7s 119ms/step - loss: 3.3574 - accuracy: 0.0631 - val_loss: 3.3289 - val_accuracy: 0.0533
Epoch 3/50
62/62 [==============================] - 7s 119ms/step - loss: 3.2330 - accuracy: 0.0823 - val_loss: 3.1852 - val_accuracy: 0.0711
Epoch 4/50
62/62 [==============================] - 7s 119ms/step - loss: 3.1473 - accuracy: 0.0974 - val_loss: 3.0204 - val_accuracy: 0.0978
Epoch 5/50
62/62 [==============================] - 7s 120ms/step - loss: 3.0464 - accuracy: 0.1100 - val_loss: 2.9840 - val_accuracy: 0.1133
Epoch 6/50
62/62 [==============================] - 7s 120ms/step - loss: 2.9722 - accuracy: 0.1312 - val_loss: 3.1045 - val_accuracy: 0.1000
Epoch 7/50
62/62 [==============================] - 7s 120ms/step - loss: 2.8958 - accuracy: 0.1348 - val_loss: 2.8926 - val_accuracy: 0.1289
Epoch 

### Check performance

In [41]:
score ,acc = model.evaluate(test_data)
print("Score is :",score)
print("Accuracy :",acc)


15/15 [==============================] - 2s 139ms/step - loss: 2.1114 - accuracy: 0.4111
Score is : 2.111417293548584
Accuracy : 0.41111111640930176


In [49]:
score ,acc = model.evaluate(train_data)
print("Score is :",score)
print("Accuracy :",acc)

62/62 [==============================] - 6s 95ms/step - loss: 1.2041 - accuracy: 0.6098
Score is : 1.2040568590164185
Accuracy : 0.6097930073738098


### Save Model

In [42]:
#save our model

model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("models.h5")

In [46]:
# Install Tensorflow.js
!pip install tensorflowjs

     |████████████████████████████████| 77 kB 3.2 MB/s 
     |████████████████████████████████| 462 kB 11.3 MB/s 


In [48]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model,'/content/drive/MyDrive/GCIT/PRJ303/Canvas_Dataset')

### Check performance

In [52]:
print("[INFO] accuracies....")
# Train accuracy
scores = model.evaluate(train_data, steps=len(train_data), verbose=1)
print("Train Accuracy: %.2f%%" % (scores[1]*100))

# Test accuracy
scores = model.evaluate(test_data, steps=len(test_data), verbose=1)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

[INFO] accuracies....
62/62 [==============================] - 6s 93ms/step - loss: 1.2354 - accuracy: 0.6058
Train Accuracy: 60.58%
15/15 [==============================] - 1s 87ms/step - loss: 2.0508 - accuracy: 0.4222
Test Accuracy: 42.22%


### Plot graph

In [51]:
import numpy as np
plt.figure(0)
plt.plot(history.history['accuracy'],'r')
plt.plot(history.history['val_accuracy'],'g')
plt.xticks(np.arange(0, 50, 2))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])

plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.xticks(np.arange(0,50, 2))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.show()

AttributeError: ignored

<Figure size 432x288 with 0 Axes>

### Evaluate the model


In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
%matplotlib inline

# Reset the validation generator and evaluate the network after fine-tuning just the network head
print("[INFO] evaluating the model...")
test_data.reset()
plt.figure(figsize=(10,10))
predIdxs = model.predict(test_data, steps = len(test_data))
predIdxs = np.argmax(predIdxs, axis=1)
matrix = confusion_matrix(test_data.classes, predIdxs)
sns.heatmap(matrix, annot=True, cbar=True, fmt='d')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

print(classification_report(test_data.classes, predIdxs, target_names=test_data.class_indices.keys()))